In [4]:
pip install flask-jwt-extended


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
from flask_jwt_extended import JWTManager, jwt_required, create_access_token

app.config['JWT_SECRET_KEY'] = 'your-secret-key'
jwt = JWTManager(app)

@app.route('/login', methods=['POST'])
def login():
    username = request.json.get('username')
    password = request.json.get('password')
    # Replace this with real authentication logic
    if username == 'admin' and password == 'password':
        access_token = create_access_token(identity={'username': username})
        return jsonify(access_token=access_token)
    else:
        return jsonify({'msg': 'Bad username or password'}), 401

@app.route('/predict', methods=['POST'])
@jwt_required()
def predict():
    try:
        # Load model and scaler
        model = joblib.load('models/random_forest_model.pkl')
        scaler = joblib.load('models/scaler.pkl')

        # Get input data
        data = request.get_json()
        input_features = np.array(data['features']).reshape(1, -1)

        # Preprocess and predict
        input_features = scaler.transform(input_features)
        prediction = model.predict(input_features)
        probability = model.predict_proba(input_features).tolist()

        return jsonify({'prediction': int(prediction[0]), 'probability': probability})

    except Exception as e:
        return jsonify({'error': str(e)})


ModuleNotFoundError: No module named 'flask_jwt_extended'

In [8]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [9]:
curl -X POST https://your-app-name.herokuapp.com/login \
-H "Content-Type: application/json" \
-d '{"username": "admin", "password": "password"}'


SyntaxError: invalid syntax (352235188.py, line 1)

In [10]:
curl -X POST https://your-app-name.herokuapp.com/predict \
-H "Content-Type: application/json" \
-H "Authorization: Bearer your_jwt_token" \
-d '{"features": [34, 1, 120, 80, 1.2, 0, 1]}'


SyntaxError: invalid syntax (1102773080.py, line 1)

In [12]:
import logging

logging.basicConfig(filename='app.log', level=logging.INFO, 
                    format='%(asctime)s:%(levelname)s:%(message)s')

@app.errorhandler(Exception)
def handle_exception(e):
    logging.error(f"An error occurred: {str(e)}")
    return jsonify({'error': 'An internal error occurred'}), 500


NameError: name 'app' is not defined